In [ ]:
from scryfall_functions import compare_strings, Query

In [ ]:
targets = []

In [ ]:
Query(targets=targets,
      details=True).execute()

In [ ]:
Query('',
      targets=targets,
      verbose=True).execute()